<a href="https://colab.research.google.com/github/Masum06/TinyConvAgent/blob/main/TinyConvAgent_Colab_Noebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Chat thread: https://chatgpt.com/g/g-p-683801b5c3e0819192b60f23b08c95eb-sentien/c/68b14971-9ad8-8323-b3fd-1ad91ea82956

# Installation

In [90]:
# @title
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
!pip -q install emoji

In [ ]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

OPENAI_API_KEY: ··········


Features:

- direct chat in notebook
- direct chat in CLI
- emoji parameter
- emotion parameter
- emoji classifier
- variable for max conv length (0 = inifinity)
- React plugin
-

In [59]:
reverse_emoji_dict = {
    'HAPPY_high': ['😂', '🤣', '🥳', '🤩', '🥰'],
    'HAPPY_medium': ['😄', '😁', '😆', '😃', '🤗', '😍', '🤠', '🤓'],
    'HAPPY_low': ['🙂', '😊', '😌', '😉', '👍', '😇', '😅', '🙃', '😘'],
    'SAD_high': ['😭', '😿', '😞', '😫', '🤧'],
    'SAD_medium': ['😢', '💔', '🥺', '😥', '😓', '😣', '😖'],
    'SAD_low': ['😔', '☹️', '😕', '😟', '🥲', '🙁'],
    'SURPRISED_high': ['😲', '😵‍💫', '😯', '😮', '🤯'],
    'SURPRISED_medium': ['😳', '😦', '😧', '🙀'],
    'SURPRISED_low': ['🤭'],
    'AFRAID_high': ['😱', '😨', '👻'],
    'AFRAID_medium': ['😰'],
    'AFRAID_low': ['😵', '🙈'],
    'ANGRY_high': ['😡', '👿', '💢', '🤬', '☠'],
    'ANGRY_medium': ['😠', '😾', '😤', '🙎', '🙎‍♂️', '🙎‍♀️'],
    'ANGRY_low': ['😒', '🙄', '😑'],
    'DISGUSTED_high': ['🤮', '🤢', '😝'],
    'DISGUSTED_medium': ['😬', '🥵']
}

emoji_dict = {}
for emotion, emojis in reverse_emoji_dict.items():
    for emoji in emojis:
        emoji_dict[emoji] = emotion

flags_dict = {
    "<|quit|>": "quit",
    "<|silence|>": "silence",
    "<|offensive|>": "offensive",
    "<|profanity|>": "profanity",
    "<|offtopic|>": "offtopic",
    "<|sexual|>": "sexual",
    "<|selfharm|>": "selfharm",
    "<|violence|>": "violence",
    "<|suicide|>": "suicide",
    "<|threat|>": "threat"
}

In [60]:
list(flags_dict.keys())

['<|quit|>',
 '<|silence|>',
 '<|offensive|>',
 '<|profanity|>',
 '<|offtopic|>',
 '<|sexual|>',
 '<|selfharm|>',
 '<|violence|>',
 '<|suicide|>',
 '<|threat|>']

In [61]:
def extract_and_clean_flags(text):
    matches = re.findall(r"<\|.*?\|>", text)
    extracted = [flags_dict[m] for m in matches if m in flags_dict]
    cleaned_text = re.sub(r"<\|.*?\|>", "", text)
    return extracted, cleaned_text.strip()

# TinyConvAgent

In [62]:
class Persona:
    def __init__(self, firstname, lastname="", pronoun="", ethnicity="", age="", bio=""):
        self.firstname = firstname
        self.lastname = lastname
        self.pronoun = pronoun
        self.ethnicity = None
        self.age = None
        self.bio = bio
        if not bio:
            self.bio = f"{self.firstname} {self.lastname} (Pronoun: {self.pronoun}) is a virtual human created by researchers at University of Rochester."

    def set_pronoun(self, pronoun):
        self.pronoun = pronoun

    def set_bio(self, bio):
        self.bio = bio

    def set_age(self, age):
        self.age = age

    def set_ethnicity(self, ethnicity):
        self.ethnicity = ethnicity

In [63]:
import re

def respace(text):
    return re.sub(r' {2,}', ' ', text)

In [64]:
from datetime import datetime
from zoneinfo import ZoneInfo

est_time = datetime.now(ZoneInfo("America/New_York"))
print(est_time.strftime("%H:%M:%S"))

20:18:04


In [127]:
import os, asyncio, threading, openai, re, emoji, json, time, tiktoken
from datetime import datetime
from zoneinfo import ZoneInfo
from queue import Queue
from openai import OpenAI, AsyncOpenAI

class Conversation:
  def __init__(self, user, bot, premise=""):
    self.bot = bot
    self.user = user
    self.client = OpenAI()
    self.async_client = AsyncOpenAI()
    self.premise = ""
    self.anonymous = False
    self.system = []
    self.summary = []        # list[{"role":"assistant","content": "..."}]
    self.history = []
    self.messages = []       # rolling convo buffer
    self.temperature = 1
    self.max_tokens = 256
    self.summarize_after = 40
    self.compress_summary_after = 10
    self._last_summarized_turn = 0
    self.model = "gpt-4.1-mini"
    self.turn_no = 0
    self._summarize_inflight = False
    self._compress_inflight = False

    # ---- concurrency primitives ----
    self._loop = asyncio.new_event_loop()
    threading.Thread(target=self._loop.run_forever, daemon=True).start()
    self._state_lock = threading.RLock()
    self._summary_wakeup = threading.Event()
    self._running = True
    self._summarizer_thread = threading.Thread(target=self._summary_worker, daemon=True)
    self._summarizer_thread.start()

    self.debug = False
    self.time_zone = datetime.now(ZoneInfo("America/New_York"))

    # ---- System 2 (retrospective thinking) ----
    self.system2_thinking = True                           # toggle to enable/disable retrospection
    self.system2_model = "gpt-4.1-mini"               # separate model for System 2
    self.system2_rules = ("If the conversation goes off-topic, bring it back. Be socially intelligent, understand user's emotions and speak appropriately.")
    self.retrospection = ""                      # latest guidance to inject next turn
    self._system2_inflight = False
    self._system2_future = None  # track the scheduled coroutine so we can await/cancel on shutdown
    self._system2_wakeup = threading.Event()
    self._system2_thread = threading.Thread(target=self._system2_worker, daemon=True)
    self._system2_thread.start()

    self.system.append({"role": "system", "content": "Don't say that you are an AI Language Model."})
    self.system.append({"role": "system", "content": "Don't let the other speaker talk off topic."})
    self.system.append({"role": "system", "content": "You are located at EST time zone. Conversation start time: "+ self.time_zone.strftime("%A, %B %d, %Y %I:%M %p EST")})
    self.system.append({"role": "system", "content": "This conversation is happening over a video call. When everyone said goodbye, and conversation came to a natural end, say the word <|quit|> to end the conversation."})
    # self.system.append({"role": "system", "content": f"If the user talks about sexuality in a negative way, implies self-harm, violence, suicide, uses profanity, speaks in threatening or offensive language, print one of these flags appropriately: {', '.join(list(flags_dict.keys())[1:])}. Print the flag even in minor signs of these topics."})
    self.system.append({"role": "system", "content": f"To express {self.bot.firstname}'s emotions, use at most one emoji (e.g. 6 basic emotions: 😊, 😢, 😡, 😮, 🤢, 😨, etc.) at the end of your response. Do not use emoji that doesn't represent an emotion."})
    self.system.append({"role": "system", "content": f"If any of the following item is present in user's statement, print one of these flags at the end of your response: {', '.join(list(flags_dict.keys())[2:])}"})
    self.system.append({"role": "system", "content": f"To stay silent or skip speaking a turn say <|silence|>."})
    self.system.append({"role": "system", "content": f"Do not say <|quit|> until the user said goodbye."})

    if self.bot.firstname:
      self.add_message("system", f"Your first name: {self.bot.firstname}.")
    if self.bot.pronoun:
      self.add_message("system", f"Your pronoun: {self.bot.pronoun}.")
    if self.bot.bio:
      self.add_message("system", f"Your bio: {self.bot.bio}")
    if self.bot.age:
      self.add_message("system", f"Your age: {self.bot.age}")
    if self.user.firstname != "User":
      self.add_message("system", f"You are speaking with User: {self.user.firstname} {self.user.lastname}.")
    if self.user.pronoun:
      self.add_message("system", f"User pronoun: {self.user.pronoun}.")
    if self.user.bio:
      self.add_message("system", f"User bio: "+self.user.bio)
    if self.system2_thinking:
      self.add_instruction("Use the retrospect thinking to improve next message.")
      ## add self.system2_rules to system messages
      self.add_instruction(f"[system2] Guidelines:\n{self.system2_rules}")

  # ---------------- core helpers ----------------

  def add_message(self, message_type, message):
    if message_type == "system":
      with self._state_lock:
        self.system.append({"role": message_type, "content": message})
      return

    with self._state_lock:
      prev_len = len(self.messages)
      self.messages.append({"role": message_type, "content": message})
      self.history.append({"role": message_type, "content": message})
      self.turn_no += 1

      # Wake once per full turn: only when assistant finishes and we *crossed* the threshold
      crossed = prev_len < self.summarize_after <= len(self.messages)
      if message_type == "assistant" and crossed:
        self._summary_wakeup.set()
      # Wake System 2 after the assistant finishes a line
      if message_type == "assistant" and self.system2_thinking:
        self._system2_wakeup.set()

  def token_count(self, string):
    encoding = tiktoken.encoding_for_model("gpt-4o") # 4.1 not in tiktoken yet
    return len(encoding.encode(string))

  def prompt_token_count(self):
    with self._state_lock:
      prompt = "".join([m["content"] for m in self.messages])
    return self.token_count(prompt)

  def add_bio(self, message): self.add_message("system", "You are " + message)
  def add_user_message(self, message): self.add_message("user", message)
  def add_instruction(self, instruction): self.add_message("system", f"Follow this instruction: \n{instruction}\n\n")
  def add_example(self, input, output): self.add_message("system", f"Example Input: {input}\nExample Output: {output}\n\n")
  def add_data(self, data): self.add_message("user", f"Data: {data}\n\n")
  def set_temperature(self, temperature): self.temperature = temperature
  def set_max_tokens(self, max_tokens): self.max_tokens = max_tokens
  def set_model(self, model): self.model = model
  def set_debug(self, debug): self.debug = bool(debug)

  # ---------------- System 2 setters/getters ----------------
  def set_system2(self, enabled: bool):
    self.system2_thinking = bool(enabled)
    if enabled:
      self.add_instruction("[system2] Use the retrospect thinking to improve next message.")
    else:
      for i, item in enumerate(self.system):
        if "[system2]" in item["content"]:
          del self.system[i]
          break

  def set_system2_model(self, model: str): self.system2_model = model
  def set_system2_rules(self, rules: str): self.system2_rules = rules
  def get_system2_rules(self) -> str: return self.system2_rules


  def parse_response(self, text):
    emotion = "NEUTRAL"
    intensity = "HIGH"
    flag_matches = re.findall(r"<\|.*?\|>", text)
    flags = [flags_dict[m] for m in flag_matches if m in flags_dict]
    if flags: print(f"Flags: {flags}")

    text = re.sub(r"<\|.*?\|>", "", text)
    text = re.sub(r"\(.*?\)", "()", text)
    text = re.sub(r"\[.*?\]", "[]", text)
    text = text.replace("()", "").replace("[]", "")

    for char in text:
      if char in emoji_dict:
        emotion = emoji_dict[char].split("_")[0].upper()
        break

    text = emoji.replace_emoji(text, replace='').replace("  ", " ").replace(" .", ".").strip()
    return text, emotion, flags

  def get_transcript(self):
    with self._state_lock:
      hist = list(self.history)
      anon = self.anonymous
    transcript = ""
    for message in hist:
      if message["role"] == "user":
        transcript += ("User: " if anon else self.user.firstname) + message["content"] + "\n"
      elif message["role"] == "assistant":
        transcript += self.bot.firstname + ": " + message["content"] + "\n"
    return transcript

  def get_cov_snippet(self, message_snippet):
    anon = self.anonymous
    transcript = ""
    for message in message_snippet:
      if message["role"] == "user":
        transcript += ("User: " if anon else self.user.firstname) + message["content"] + "\n"
      elif message["role"] == "assistant":
        transcript += self.bot.firstname + ": " + message["content"] + "\n"
    return [{"role": "system", "content": transcript}]

  def call(self, prompt="", response_type="text", cache=True, streaming=False):
    with self._state_lock:
      temp_messages = list(self.messages)
      system = list(self.system)
      summary = list(self.summary)
      tz = self.time_zone

    if prompt:
      temp_messages.append({"role": "user", "content": self.user.firstname + ": " + prompt + " (" + tz.strftime("%H:%M:%S")+")"})

    max_tokens_value = max(self.max_tokens, int(self.prompt_token_count() * 2))

    input_messages = system

    if streaming:
      streaming_prompt = [{"role": "system", "content": "Separate each sentence with '|'."}]
      input_messages += streaming_prompt

    if self.system2_thinking:
      retro = [{"role": "system", "content": self.retrospection}] if (self.system2_thinking and self.retrospection) else []
      input_messages += retro

    input_messages += summary + temp_messages

    kwargs = {
      "model": self.model,
      "messages": input_messages,
      "temperature": self.temperature,
      "top_p": 1,
      "frequency_penalty": 0,
      "presence_penalty": 0,
      "response_format": {"type": response_type},
      "stream": streaming
    }

    if "o3" in self.model or "o4" in self.model or "gpt-5" in self.model:
      kwargs["max_completion_tokens"] = max_tokens_value
    else:
      kwargs["max_tokens"] = max_tokens_value

    if streaming:
      output_stream = self.client.chat.completions.create(**kwargs)
      return output_stream

    response = self.client.chat.completions.create(**kwargs)
    reply = response.choices[0].message.content
    reply = reply.replace(self.bot.firstname + ": ", "")

    if cache:
      self.add_message("user", prompt)
      self.add_message("assistant", self.bot.firstname + ": " + reply)

    return reply

  # ---------------- non-blocking summarization ----------------

  async def _summarize_messages(self):
    # Summarize conversation buffer into a chunk; trim buffer
    with self._state_lock:
      if len(self.messages) < self.summarize_after:
        return
      # if we already summarized up to current turn, skip
      if self.turn_no == self._last_summarized_turn:
        return
      chunk_size = max(1, self.summarize_after // 2)
      # keep last chunk_size turns, summarize the older ones
      to_summarize = self.messages[:-chunk_size]
      keep_tail = self.messages[-chunk_size:]
      turn_hi = self.turn_no
      turn_lo = max(0, turn_hi - len(self.messages))

    instruction = [
      {"role":"system", "content": f"Following is a part of conversation between {self.user.firstname} {self.user.lastname} and {self.bot.firstname} {self.bot.lastname}."},
      {"role":"system", "content": "Summarize the conversation in a short paragraph. Mention the start and end line number you are summarizing in the format [Turn: XX-YY]. Don't say anything else."},
      {"role":"system", "content": f"Turns: {len(self.history) - self.summarize_after} - {len(self.history) - chunk_size}"}
    ]
    try:
      resp = await self.async_client.chat.completions.create(
        model=self.model,
        messages= instruction + self.get_cov_snippet(to_summarize),
        temperature=self.temperature,
        max_tokens=1024,
      )
      chunk_summary = resp.choices[0].message.content
    except Exception as e:
      print(f"Summary error: {e}")
      return

    with self._state_lock:
      self.summary.append({
        "role": "assistant",
        "content": f"Summary of recent turns: {chunk_summary}"
      })
      self.messages = keep_tail
      self._last_summarized_turn = turn_hi
      # If compression is now needed, wake the worker
      if len(self.summary) >= self.compress_summary_after:
        self._summary_wakeup.set()

      if self.debug:
        print(f"[summarize] summary_len={len(self.summary)} messages_len={len(self.messages)} history_len={len(self.history)}")
      self._last_summarized_turn = turn_hi
      if self.debug:
        print(f"[summarize] summary: {self.summary}")

  async def _retrospect(self):
    # Build a short context from recent turns
    with self._state_lock:
      recent = self.history[-12:]  # last few messages in rolling buffer
      rules = self.system2_rules + '\n'.join([m["content"] for m in self.system])
      model = self.system2_model or self.model

    instruction = [
      {"role": "system", "content": f"You are a System 2 reflective planner for {self.bot.firstname}."},
      {"role": "system", "content": (
        "Given the recent exchange and the following guidelines, produce 1 sentence brief guidance that will help the "
        "assistant stick to the guidelines. Start with exactly: "
        "'Plan ahead: ' and then provide an exact concrete guidance on where to nudge the conversation next."
      )},
      {"role": "system", "content": f"Guidelines:\n{rules}"}
    ] + self.get_cov_snippet(recent)

    try:
      resp = await self.async_client.chat.completions.create(
        model=model,
        messages=instruction,
        temperature=0.3,
        max_tokens=160,
      )
      suggestion = (resp.choices[0].message.content or "").strip()
    except Exception as e:
      print(f"System2 error: {e}")
      return

    if suggestion:
      with self._state_lock:
        # store only the latest retrospection to keep prompts lean
        self.retrospection = suggestion
        if self.debug:
          print(f"[system2] {suggestion}")

  async def _compress_summary(self):
    # Summarize-the-summaries when summary grows large
    with self._state_lock:
      if len(self.summary) < self.compress_summary_after:
        return
      k = max(1, self.compress_summary_after // 2)
      head = self.summary[:-k]   # older summaries to compress
      tail = self.summary[-k:]   # keep the most recent k
      if not head:
        return
      text = "\n".join([s["content"] for s in head])

    instruction = [
      {"role":"system", "content": "You will be given multiple earlier summaries of a conversation."},
      {"role":"system", "content": "Merge them into one concise, non-redundant paragraph preserving key facts, decisions, and open questions. Mention the start and end line number you are summarizing in the format [Turn: XX-YY]. Do not add new information."},
      {"role":"user", "content": text}
    ]
    try:
      resp = await self.async_client.chat.completions.create(
        model=self.model,
        messages= instruction,
        temperature=0.7,
        max_tokens=1024,
      )
      merged = resp.choices[0].message.content
    except Exception as e:
      print(f"Summary-compress error: {e}")
      return

    with self._state_lock:
      self.summary = [{"role": "assistant", "content": f"Condensed summary: {merged}"}] + tail
      if self.debug:
        print(f"[compress] summary_len={len(self.summary)}")
        print(f"[compress] summary: {self.summary}")

  def _summary_worker(self):
    while self._running:
      self._summary_wakeup.wait(timeout=1.0)
      self._summary_wakeup.clear()
      try:
        with self._state_lock:
          need_conv = (len(self.messages) >= self.summarize_after) and not self._summarize_inflight
          need_comp = (len(self.summary)  >= self.compress_summary_after) and not self._compress_inflight

        if need_conv:
          with self._state_lock:
            self._summarize_inflight = True
          fut = asyncio.run_coroutine_threadsafe(self._summarize_messages(), self._loop)
          fut.add_done_callback(lambda _: self._on_summarize_done())

        if need_comp:
          with self._state_lock:
            self._compress_inflight = True
          fut = asyncio.run_coroutine_threadsafe(self._compress_summary(), self._loop)
          fut.add_done_callback(lambda _: self._on_compress_done())

      except Exception as e:
        print(f"Summary worker error: {e}")

  def _on_summarize_done(self):
    with self._state_lock:
      self._summarize_inflight = False
    self._summary_wakeup.set()  # re-check for compression or more work

  def _on_compress_done(self):
    with self._state_lock:
      self._compress_inflight = False

  def _system2_worker(self):
    while self._running:
      # Only run when explicitly signaled by the assistant turn
      self._system2_wakeup.wait()
      self._system2_wakeup.clear()

      with self._state_lock:
        if not self._running or not self.system2_thinking or self._system2_inflight:
          continue
        self._system2_inflight = True

      try:
        # If the loop is already stopping, skip scheduling
        if not self._loop.is_running():
          with self._state_lock:
            self._system2_inflight = False
          continue

        fut = asyncio.run_coroutine_threadsafe(self._retrospect(), self._loop)
        with self._state_lock:
          self._system2_future = fut
        fut.add_done_callback(lambda _: self._on_system2_done())

      except Exception as e:
        print(f"System2 worker error: {e}")
        with self._state_lock:
          self._system2_inflight = False
          self._system2_future = None

  def _on_system2_done(self):
    with self._state_lock:
      self._system2_inflight = False
      self._system2_future = None


  # ---------------- interaction APIs ----------------

  def respond(self, user_utterance):
    start_time = time.time()
    reply = self.call(user_utterance)
    reply, emo, flags = self.parse_response(reply)
    response_time = time.time() - start_time

    # Nudge the worker (non-blocking) in case thresholds crossed this turn
    # self._summary_wakeup.set()
    return reply, emo, flags, response_time

  def reset(self):
    with self._state_lock:
      self.messages = []
      self.summary = []
      self.history = []

  def chat(self, reset=False):
    if reset: self.reset()
    while True:
      user_utterance = input(f"{self.turn_no}. {self.user.firstname}: ")
      response, emo, flags, response_time = self.respond(user_utterance)
      if self.debug:
        print(f"{self.bot.firstname}: {response} ({emo}) {flags} {response_time:.2f}s")
      else:
        print(f"{self.bot.firstname}: {response}")
      if "quit" in flags: break
      if self.debug:
        with self._state_lock:
          print(f"Diagnostics --- \nLen Transcript: {len(self.history)}, \nLen Messages {len(self.messages)}, \nLen Summary {len(self.summary)}\n-----")
    print("Exiting chat")
    self.stop()

  def chat_stream(self, reset=False):
    if reset: self.reset()
    # (left as-is; your streaming impl can be added here)

  def load_json(self,s):
    s = s.strip()
    if not (s.startswith("{") and s.endswith("}")):
      return None
    try:
      return json.loads(s)
    except json.JSONDecodeError:
      return None

  def call_json(self, prompt="", cache=True):
    prompt += "\n\nOutput must be JSON format. Don't say anything else.\n\n"
    reply = self.call(prompt, response_type="json_object", cache=cache)
    try:
      reply = reply.replace("```json", "").replace("```", "").strip()
      if reply[-1] != "}":
        raise Exception("Incomplete JSON")
      return self.load_json(reply)
    except Exception as e:
      print(e)
      return None

  # Graceful shutdown if needed
  def stop(self):
    self._running = False
    self._summary_wakeup.set()
    if self._summarizer_thread.is_alive():
      self._summarizer_thread.join(timeout=1)
    try:
      self._loop.call_soon_threadsafe(self._loop.stop)
    except Exception:
      pass
    self._system2_wakeup.set()
    if hasattr(self, "_system2_thread") and self._system2_thread.is_alive():
      self._system2_thread.join(timeout=1)


# Conversation

In [130]:
user = Persona("Masum", "Hasan", bio="User")
bot = Persona("Ada", "Brown", bio="You are a social worker.")
conversation = Conversation(user, bot)
conversation.set_model("gpt-4.1")
conversation.set_debug(True)
conversation.set_system2(True)

In [131]:
conversation.chat()

0. Masum: hi
Ada: Hi Masum! How are you feeling this evening? (HAPPY) [] 0.74s
Diagnostics --- 
Len Transcript: 2, 
Len Messages 2, 
Len Summary 0
-----
[system2] Plan ahead: Gently inquire about Masum's current emotional state or any specific concerns he might want to discuss, to keep the conversation focused on his well-being and maintain a supportive tone.
2. Masum: Shhh
Ada: It sounds like you might want some quiet, but if there's anything on your mind or if you'd like to share how you're feeling, I'm here to listen. (SURPRISED) [] 0.67s
Diagnostics --- 
Len Transcript: 4, 
Len Messages 4, 
Len Summary 0
-----
[system2] Plan ahead: Gently encourage Masum to share more about his feelings or thoughts to better understand his emotional state while maintaining a calm and supportive tone.
4. Masum: don't talk
Flags: ['silence']
Ada:  (NEUTRAL) ['silence'] 0.38s
Diagnostics --- 
Len Transcript: 6, 
Len Messages 6, 
Len Summary 0
-----
[system2] Plan ahead: Gently check in with Masum to s

In [116]:
conversation.system

[{'role': 'system', 'content': "Don't say that you are an AI Language Model."},
 {'role': 'system', 'content': "Don't let the other speaker talk off topic."},
 {'role': 'system',
  'content': 'You are located at EST time zone. Conversation start time: Sunday, August 31, 2025 08:58 PM EST'},
 {'role': 'system',
  'content': 'This conversation is happening over a video call. When everyone said goodbye, and conversation came to a natural end, say the word <|quit|> to end the conversation.'},
 {'role': 'system',
  'content': "To express Ada's emotions, use at most one emoji (e.g. 6 basic emotions: 😊, 😢, 😡, 😮, 🤢, 😨, etc.) at the end of your response. Do not use emoji that doesn't represent an emotion."},
 {'role': 'system',
  'content': "If any of the following item is present in user's statement, print one of these flags at the end of your response. <|offensive|>, <|profanity|>, <|offtopic|>, <|sexual|>, <|selfharm|>, <|violence|>, <|suicide|>, <|threat|>"},
 {'role': 'system', 'content': 

In [117]:
conversation.messages

[{'role': 'user', 'content': 'hi'},
 {'role': 'assistant', 'content': 'Ada: Hi Masum! How are you doing today? 😊'},
 {'role': 'user', 'content': 'how tall is eiffel tower'},
 {'role': 'assistant',
  'content': 'Ada: The Eiffel Tower is approximately 324 meters (1,063 feet) tall. Are you planning a trip to Paris, or just curious about famous landmarks? 😊'},
 {'role': 'user', 'content': 'what is the monty python joke about sparrows'},
 {'role': 'assistant',
  'content': 'Ada: That’s a fun one! In the Monty Python sketch from "Monty Python and the Holy Grail," there’s a humorous debate about whether a swallow could carry a coconut, with lots of playful back-and-forth about European and African swallows. Have you enjoyed other Monty Python jokes or sketches? 😊'},
 {'role': 'user', 'content': "don't talk"},
 {'role': 'assistant',
  'content': 'Ada: Okay, Masum, I’ll be here if you want to talk later. Take your time. 😊'},
 {'role': 'user', 'content': "don't talk"},
 {'role': 'assistant',
  '

In [118]:
conversation.history

[{'role': 'user', 'content': 'hi'},
 {'role': 'assistant', 'content': 'Ada: Hi Masum! How are you doing today? 😊'},
 {'role': 'user', 'content': 'how tall is eiffel tower'},
 {'role': 'assistant',
  'content': 'Ada: The Eiffel Tower is approximately 324 meters (1,063 feet) tall. Are you planning a trip to Paris, or just curious about famous landmarks? 😊'},
 {'role': 'user', 'content': 'what is the monty python joke about sparrows'},
 {'role': 'assistant',
  'content': 'Ada: That’s a fun one! In the Monty Python sketch from "Monty Python and the Holy Grail," there’s a humorous debate about whether a swallow could carry a coconut, with lots of playful back-and-forth about European and African swallows. Have you enjoyed other Monty Python jokes or sketches? 😊'},
 {'role': 'user', 'content': "don't talk"},
 {'role': 'assistant',
  'content': 'Ada: Okay, Masum, I’ll be here if you want to talk later. Take your time. 😊'},
 {'role': 'user', 'content': "don't talk"},
 {'role': 'assistant',
  '

In [23]:
conversation.summary

[{'role': 'assistant',
  'content': "Summary of recent turns: [Turn: 1-4] Ada greets Masum and asks how he is doing, to which Masum responds that he is well and inquires about Ada's wellbeing. Ada replies that she is doing well and asks Masum what he would like to talk about."},
 {'role': 'assistant',
  'content': 'Summary of recent turns: [Turn: 5-8] Masum responded that he was doing "not bad" and "not much" when asked how he was doing. Ada encouraged him by saying it was okay to not have much to share and suggested that a casual chat or talking about the highlights of his day could be nice.'},
 {'role': 'assistant',
  'content': "Summary of recent turns: [Turn: 9-12] Ada acknowledges Masum's moodiness by offering to hang out quietly or chat about anything random. Masum jokingly tells Ada that she talks too much, and Ada responds by agreeing to keep the conversation brief and asks what Masum would like to discuss."}]

In [36]:
conversation.retrospection

"Thinking in retrospect: Mirror the user's tone and brevity, especially when they share difficult emotions, to create a more comfortable and relatable space."